1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [2]:
def converter(x,choice):
    if(choice == "dtob"):
        res = bin(x)
    elif (choice == "btod"):
        res = int(str(x), 2)
    elif (choice == "dtoh"):
        res = hex(x)
    elif (choice == "htod"):
        res = int(str(x),16)
    elif (choice == "btoh"):
        aux = converter(x,"btod") #first convert in decimal
        res = converter(aux,"dtoh") #then in hex
    elif (choice == "htob"):
        aux = converter(x,"htod") #first convert in decimal
        res = converter(aux,"dtob")#then in bin
    return res


print(converter(10,"dtob"))
print(converter(1010,"btod"))
print(converter(10,"dtoh"))
print(converter("A","htod"))
print(converter(1010,"btoh"))
print(converter("A","htob"))

0b1010
10
0xa
10
0xa
0b1010


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [3]:
def wordToFloat(x):
    x = str(x)
    if(len(x) != 32):
        print("length must be 32 bits")
        return False
    
    sign = x[0]
    exp = x[1:9]
    mant = x[9:]
    
    print("\nsign, exponent and mantissa in binary:")
    print(sign, exp, mant)
    
    #decimal conversions for sign and exponent:
    sign = int(sign,2)
    exp = int(exp,2) - 127 #excess 127
    
    #decimal conversion for the mantissa:
    dmant = 1
    for i in range(0,len(mant)):
        dmant += 2**(-(i+1))*int(mant[i],2)
    
    print("\nsign, exponent and mantissa in decimal:")
    print(sign,exp,dmant)   
    
    res = (-1)**sign * (dmant*2**exp)
    return res

w1 = "11000000101100000000000000000000" #f.p. single precision representation for -5,500000
print("\nresult: " + str(wordToFloat(w1)))
w2 = "00000011111000000000000000000000" #f.p. single precision representation for 1,316554 * 10**(-36)
print("\nresult: " + str(wordToFloat(w2)))


sign, exponent and mantissa in binary:
1 10000001 01100000000000000000000

sign, exponent and mantissa in decimal:
1 2 1.375

result: -5.5

sign, exponent and mantissa in binary:
0 00000111 11000000000000000000000

sign, exponent and mantissa in decimal:
0 -120 1.75

result: 1.316553672920962e-36


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [4]:
a = 1.0
b = 1.0
aux = 2.0
zero = 0.0

while 1==1:
    aux = a*2
    if aux == float("inf"):
        break
    a = aux
    
while 1==1:
    aux = b/2
    if aux.hex() == zero.hex():
        break
    b = aux

print("max value until overflow is: " + str(a))

print("min value until underflow is: " + str(b))

max value until overflow is: 8.98846567431158e+307
min value until underflow is: 5e-324


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [5]:
a = 1.0
e = 1
one = 1.0

while 1==1:
    a = a + 1*10**-e #adding a small number to a
    if(a.hex() == one.hex()):
        break
    a = 1.0 #reset a
    e += 1 #increasing the exponent
    
print("min value until having no effect on the number is: " + str(1*10**-e))

min value until having no effect on the number is: 1e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [6]:
import math

def quadratic_eq(a,b,c):
    x1 = (-b + math.sqrt(b**2-4*a*c)) / (2*a)
    x2 = (-b - math.sqrt(b**2-4*a*c)) / (2*a)
    return (x1,x2)

def quadratic_eq_mult(a,b,c):
    to_multiply_to_x1 = -b - math.sqrt(b**2 - 4*a*c)
    to_multiply_to_x2 = -b + math.sqrt(b**2 - 4*a*c)
    
    print("\nHere are the factors to multiply to both numerator and denominator of x1 and x2:")
    print(to_multiply_to_x1,to_multiply_to_x2)
    
    x1 = ((-b + math.sqrt(b**2-4*a*c))*to_multiply_to_x1) / ((2*a)*to_multiply_to_x1)
    
    x2 = ((-b - math.sqrt(b**2-4*a*c))*to_multiply_to_x2) / ((2*a)*to_multiply_to_x2)
    
    return (x1,x2)

def quadratic_eq_mult_correct(a,b,c):
    to_multiply_to_x1 = -b - math.sqrt(b**2 - 4*a*c)
    to_multiply_to_x2 = -b + math.sqrt(b**2 - 4*a*c)
    
    to_multiply_to_x1_directly = to_multiply_to_x1 / to_multiply_to_x1
    to_multiply_to_x2_directly = to_multiply_to_x2 / to_multiply_to_x2
    
    print("\nHere are the factors to multiply DIRECTLY to x1 and x2:")
    print(to_multiply_to_x1_directly, to_multiply_to_x2_directly)
    
    x1 = (-b + math.sqrt(b**2-4*a*c)) / (2*a)
    x1 = x1 * to_multiply_to_x1_directly
    x2 = (-b - math.sqrt(b**2-4*a*c)) / (2*a)
    x2 = x2 * to_multiply_to_x2_directly
    
    return (x1,x2)

def quadratic_eq_correct(a,b,c):
    delta = b**2-4*a*c
    
    if(delta < 0):
        return "Negative delta -> No results in real numbers"        
    
    x1 = (-b + math.sqrt(delta)) / (2*a)
    x2 = (-b - math.sqrt(delta)) / (2*a)
    return (x1,x2)

res1 = quadratic_eq(0.001,1000,0.001)
print("res1 = " + str(res1))

res2 = quadratic_eq_mult(0.001,1000,0.001)
print("\nres2 = " + str(res2))

print("\nNotice that x1 remained unchanged, while x2 is a little bit more (exactly 1*10**-10)")
print("This is due to the order of operations made by the computer, and the fact that float have limited precision")
print("In fact, if we compute the fraction composed by -b\mp\sqrt{b^2-4ac} both at the numerator and the denominator first, we can notice that the result of this fraction is 1.0")

res3 = quadratic_eq_mult_correct(0.001,1000,0.001)
print("\nres3 = " + str(res3))

print("\nLet's now use a function that computes delta first, in order to determine if there are no solutions in real numbers")

res4 = quadratic_eq_correct(1,1,1)
print("\nres4 = " + str(res4))

res5 = quadratic_eq_correct(0.001,1000,0.001)
print("\nres5 = " + str(res5))

res1 = (-9.999894245993346e-07, -999999.999999)

Here are the factors to multiply to both numerator and denominator of x1 and x2:
-1999.999999998 -1.999978849198669e-09

res2 = (-9.999894245993346e-07, -999999.9999990001)

Notice that x1 remained unchanged, while x2 is a little bit more (exactly 1*10**-10)
This is due to the order of operations made by the computer, and the fact that float have limited precision
In fact, if we compute the fraction composed by -b\mp\sqrt{b^2-4ac} both at the numerator and the denominator first, we can notice that the result of this fraction is 1.0

Here are the factors to multiply DIRECTLY to x1 and x2:
1.0 1.0

res3 = (-9.999894245993346e-07, -999999.999999)

Let's now use a function that computes delta first, in order to determine if there are no solutions in real numbers

res4 = Negative delta -> No results in real numbers

res5 = (-9.999894245993346e-07, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [7]:
f = lambda x: x*(x-1)
der = lambda x,d: (f(x+d)-f(x))/d
der_analytical = lambda x: 2*x-1

print("derivative calculated analytically is = " + str(der_analytical(1)))
print("derivative calculated with delta = " + str(10**(-2)) + " is = " + str(der(1,10**(-2))))

print("\nNotice that the two differs because in the analytical derivative delta tends to zero")

for e in [4,6,8,10,12,14]:
    print("derivative calculated with delta = " + str(10**(-e)) + " is = " + str(der(1,10**(-e))))

print("\nNotice that the maximum accuracy is obtained with delta = 1e-08, then , the lower delta, the less accuracy")



derivative calculated analytically is = 1
derivative calculated with delta = 0.01 is = 1.010000000000001

Notice that the two differs because in the analytical derivative delta tends to zero
derivative calculated with delta = 0.0001 is = 1.0000999999998899
derivative calculated with delta = 1e-06 is = 1.0000009999177333
derivative calculated with delta = 1e-08 is = 1.0000000039225287
derivative calculated with delta = 1e-10 is = 1.000000082840371
derivative calculated with delta = 1e-12 is = 1.0000889005833413
derivative calculated with delta = 1e-14 is = 0.9992007221626509

Notice that the maximum accuracy is obtained with delta = 1e-08, then , the lower delta, the less accuracy


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [8]:
import time

def riemann_integral(N):
    y = lambda x: math.sqrt(1-x**2)

    res = 0;
    x = -1 #starting value for x in the riemann series

    for i in range(1,N+1):
        res += y(x) #compute the function at x and add it to the res of the series
        x += 2/N #add a slice to x
    res = res * (2/N) #multiply the result by the width of each slice and obtain the integral
    
    return res

for n in [100,1000,10000,100000,1000000,2000000,10000000]:
    start = time.time()
    res = riemann_integral(n) #compute integral
    end = time.time()
    print("result of the integral using Reimann with N = " + str(n) + " = " + str(res) + " time for computation = " + str(end-start))

print("\nWe can notice that using N=100 the time for computing the integral with this algorithm is almost a second")
print("The greater is N the more accurate is the result, being more similar to the analytical result")

result of the integral using Reimann with N = 100 = 1.5691342555492493 time for computation = 0.0
result of the integral using Reimann with N = 1000 = 1.5707437385010707 time for computation = 0.0019960403442382812
result of the integral using Reimann with N = 10000 = 1.5707946637153933 time for computation = 0.008994340896606445
result of the integral using Reimann with N = 100000 = 1.5707962742024018 time for computation = 0.07624030113220215
result of the integral using Reimann with N = 1000000 = 1.5707963251473536 time for computation = 0.7573199272155762
result of the integral using Reimann with N = 2000000 = 1.5707963262011113 time for computation = 1.2384376525878906
result of the integral using Reimann with N = 10000000 = 1.5707963267013612 time for computation = 6.245028018951416

We can notice that using N=100 the time for computing the integral with this algorithm is almost a second
The greater is N the more accurate is the result, being more similar to the analytical result